# Task 4 AML : Sleeping Staging

In [32]:
import sys
!{sys.executable} -m pip install mne

The directory '/home/atroska/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/atroska/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 19.0.3, however version 20.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import psd_welch
from mne.time_frequency import tfr_stockwell
from mne import EpochsArray,create_info
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import balanced_accuracy_score

In [34]:
train_eeg1 = pd.read_csv("train_eeg1.csv")
train_eeg2 = pd.read_csv("train_eeg2.csv")
train_emg = pd.read_csv("train_emg.csv")
train_labels = pd.read_csv("train_labels.csv")

test_eeg1 = pd.read_csv("test_eeg1.csv")
test_eeg2 = pd.read_csv("test_eeg2.csv")
test_emg = pd.read_csv("test_emg.csv")

sample = pd.read_csv("sample.csv")

In [35]:
train_eeg1_numpy=np.expand_dims(train_eeg1.to_numpy()[:,1:],-1)
train_eeg2_numpy=np.expand_dims(train_eeg2.to_numpy()[:,1:],-1)
train_emg_numpy=np.expand_dims(train_emg.to_numpy()[:,1:],-1)
train_labels_numpy=train_labels.to_numpy()[:,1:]

test_eeg1_numpy=np.expand_dims(test_eeg1.to_numpy()[:,1:],-1)
test_eeg2_numpy=np.expand_dims(test_eeg2.to_numpy()[:,1:],-1)
test_emg_numpy=np.expand_dims(test_emg.to_numpy()[:,1:],-1)

In [36]:
train_all_channels=np.concatenate((train_eeg1_numpy,train_eeg2_numpy,train_emg_numpy),axis=2)
train_all_channels.shape

(64800, 512, 3)

In [37]:
test_all_channels=np.concatenate((test_eeg1_numpy,test_eeg2_numpy,test_emg_numpy),axis=2)
test_all_channels.shape

(43200, 512, 3)

In [38]:
train_labels_numpy.shape

(64800, 1)

In [39]:
train_labels_numpy[np.where(train_labels_numpy==1)]=0
train_labels_numpy[np.where(train_labels_numpy==2)]=1
train_labels_numpy[np.where(train_labels_numpy==3)]=2

In [40]:
count1 = sum(map(lambda x : x == 1, train_all_channels))
count2 = sum(map(lambda x : x == 2, train_all_channels))
count3 = sum(map(lambda x : x == 3, train_all_channels))
print('there is {} one, {} two and {}3'.format(count1, count2, count3))

there is [[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]] one, [[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]] two and [[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]3


In [45]:
def frequencies(epochs, pick="eeg"):
    WAVEFORMS = {"delta": [0.5, 4],
                 "theta": [4, 7],
                 "alpha":[8, 12],
                 "sigma": [12, 16],
                 "beta": [13, 30]
                }
    psds, freqs = psd_welch(epochs, picks=pick, fmin=0.5, fmax=30)
    psds /= np.sum(psds, axis=-1, keepdims=True)
    results = []
    for fmin, fmax in WAVEFORMS.values():
        band = psds[:, :, (freqs < fmax) & (freqs >= fmin)].mean(axis=-1)
        results.append(band.reshape(len(psds), -1))
    results = np.nan_to_num(np.concatenate(results, axis=1))
    return results
    

In [42]:
names=["eeg1", "eeg2", "emg"]
types=["eeg", "eeg", "emg"]
frequence = 128
info=create_info(ch_names=names, sfreq=frequence, ch_types=types)
epochs=EpochsArray(np.reshape(train_all_channels,(64800,3, 512)), info=info)



Not setting metadata
Not setting metadata
64800 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


In [46]:
pipeline = make_pipeline(FunctionTransformer(frequencies, 
                                         validate=False),
                     RandomForestClassifier(n_estimators=200, 
                                            random_state=84,
                                            max_depth=30))

pipeline.fit(epochs, train_labels_numpy)

preds = pipeline.predict(epochs)
accuracy = balanced_accuracy_score(preds, train_labels_numpy)
print("Accuracy score: {}".format(accuracy))

Effective window size : 2.000 (s)


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Effective window size : 2.000 (s)
Accuracy score: 0.9999877152895507


In [47]:
print(preds)

[1 1 1 ... 0 0 0]


In [48]:
epochs_test=EpochsArray(np.reshape(test_all_channels,(43200,3,512)), info=info)
preds_test = pipeline.predict(epochs_test)
preds_test=preds_test+1
test_id = sample.to_numpy()[:,0]
submission={"id": test_id,
           "y": preds_test.flatten()}
submission=pd.DataFrame(submission)
print(submission.head())
submission.to_csv("y_test_1.csv")

Not setting metadata
Not setting metadata
43200 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Effective window size : 2.000 (s)
   id  y
0   0  1
1   1  1
2   2  1
3   3  1
4   4  1


In [50]:
for i in preds_test:
    if i == 1:
        print("un")
    if i == 2:
        print("deux")
    if i == 3:
        print("trois")

un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
deux
un

un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
deux
deux
trois
deux
un
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
un
un
trois
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux

deux
deux
un
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
un
deux
un
un
un
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
un
trois
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
un
deux
trois
un
un
trois
trois
trois
un
deux
trois
trois
trois
deux
trois
trois
deux
deux
un
un
un
deux
trois
deux
deux
deux
deux
deux
deux
deux
un
trois
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deu

deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
deux
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
trois
un
un
un
un
un
un
un
un
un
un
un
un
deux
deux
un
un
un
trois
deux
deux
un
un
un
un
un
un
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
d

deux
deux
deux
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
deux
un
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux


...

un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
trois
un
un
un
un
un
un
un
un
un
deux
deux
deux
deux
un
trois
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux

deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
deux
un
un
un
un
deux
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
trois
trois
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
u

deux
trois
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
trois
trois
deux
trois
deux
deux
un
trois
un
deux
deux
deux
deux
un
un
un
un
un
un
un
un
un
un
deux
deux
un
un
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
u

deux
deux
deux
deux
deux
deux
deux
deux
trois
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
trois
deux
deux
trois
trois
trois
trois
trois
trois
deux
un
trois
un
un
trois
trois
un
deux
trois
un
trois
deux
trois
un
un
deux
deux
un
un
deux
deux
deux
deux
un
un
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
trois
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
u

un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
un
deux
trois
deux
deux
un
deux
un
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
un
un
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
trois
trois
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
un
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
deux
de